In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=847f2cf4a8730312ca056b3742cfe118af89f4b71208594badcee5e24f28525d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm


In [5]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:500]")

print(dataset)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 500
})


In [6]:
len(dataset['highlights'][2])

224

In [7]:
# Extract articles and highlights
data = [(entry['article'], entry['highlights']) for entry in dataset]
# data

In [8]:
# import re
# import numpy as np
# from collections import Counter, defaultdict
# from sklearn.model_selection import train_test_split

# # # Sample text data for text summarization
# # data = [
# #     ("The cat sat on the mat.", "The cat is on the mat."),
# #     ("Transformers are powerful models.", "Transformers are powerful."),
# #     ("I love programming.", "Programming is my passion."),
# #     ("Byte pair encoding is useful.", "BPE is useful."),
# # ]

# # Define special tokens
# SPECIAL_TOKENS = {
#     "<PAD>": 0,
#     "<SOS>": 1,
#     "<EOS>": 2,
#     "<UNK>": 3,  # For unknown words
# }

# # Tokenizer: Byte Pair Encoding
# def get_initial_vocab(corpus):
#     """
#     Create an initial vocabulary with all unique characters.
#     """
#     vocab = defaultdict(int)
#     for sentence in corpus:
#         for word in sentence.split():
#             vocab[" ".join(list(word)) + " </w>"] += 1  # Add word boundary
#     return vocab

# def merge_vocab(vocab):
#     """
#     Find the most frequent pair and merge it into a new token.
#     """
#     pairs = Counter()
#     for word, freq in vocab.items():
#         tokens = word.split()
#         for i in range(len(tokens) - 1):
#             pairs[(tokens[i], tokens[i + 1])] += freq
    
#     if not pairs:
#         return None
    
#     most_common = max(pairs, key=pairs.get)
#     return most_common

# def apply_merge_rule(vocab, pair):
#     """
#     Apply the merge rule to the vocabulary.
#     """
#     new_vocab = {}
#     bigram = " ".join(pair)
#     replacement = "".join(pair)
    
#     for word, freq in vocab.items():
#         new_word = re.sub(rf"{re.escape(bigram)}", replacement, word)
#         new_vocab[new_word] = freq
    
#     return new_vocab

# def byte_pair_encoding(corpus, num_merges=20):
#     """
#     Perform BPE on the given corpus and return the vocabulary.
#     """
#     vocab = get_initial_vocab(corpus)
#     for _ in range(num_merges):
#         pair = merge_vocab(vocab)
#         if not pair:
#             break
#         vocab = apply_merge_rule(vocab, pair)
#     return vocab

# # Generate BPE vocabulary
# all_texts = [src for src, tgt in data] + [tgt for src, tgt in data]
# bpe_vocab = byte_pair_encoding(all_texts, num_merges=150)

# # Map tokens to unique IDs
# def build_vocab_from_bpe(bpe_vocab):
#     vocab = dict(SPECIAL_TOKENS)  # Start with special tokens
#     index = len(SPECIAL_TOKENS)
#     for token in bpe_vocab.keys():
#         for sub_token in token.split():
#             if sub_token not in vocab:
#                 vocab[sub_token] = index
#                 index += 1
#     return vocab

# vocab = build_vocab_from_bpe(bpe_vocab)
# print(len(vocab))
# # Tokenization
# def tokenize_with_bpe(text, vocab):
#     """
#     Tokenize text using BPE rules and convert to IDs.
#     """
#     words = text.split()
#     tokenized = []
#     for word in words:
#         chars = " ".join(list(word)) + " </w>"
#         for pair in sorted(vocab, key=len, reverse=True):
#             chars = chars.replace(pair, pair.replace(" ", ""))
#         tokenized.extend(chars.split())
#     return [vocab.get(token, vocab["<UNK>"]) for token in tokenized]

# # Padding and preparation
# def pad_sequences(sequences, max_len, pad_token_id):
#     return np.array([
#         seq + [pad_token_id] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]
#         for seq in sequences
#     ])

# # In your data preparation or DataLoader creation
# def prepare_data(data, vocab, max_len=128):
#     input_ids, target_ids = [], []
#     input_attention_masks, target_attention_masks = [], []
    
#     for src, tgt in data:
#         # Tokenize and encode source and target
#         src_ids = tokenize_with_bpe(src, vocab)
#         tgt_ids = tokenize_with_bpe(tgt, vocab)
        
#         # Add special tokens
#         src_ids = [vocab["<SOS>"]] + src_ids + [vocab["<EOS>"]]
#         tgt_ids = [vocab["<SOS>"]] + tgt_ids + [vocab["<EOS>"]]
        
#         # Create 3D attention masks
#         src_mask = [1 if token != vocab["<PAD>"] else 0 for token in src_ids]
#         tgt_mask = [1 if token != vocab["<PAD>"] else 0 for token in tgt_ids]
        
#         # Pad sequences and masks
#         src_ids = src_ids + [vocab["<PAD>"]] * (max_len - len(src_ids))[:max_len]
#         tgt_ids = tgt_ids + [vocab["<PAD>"]] * (max_len - len(tgt_ids))[:max_len]
#         src_mask = src_mask + [0] * (max_len - len(src_mask))[:max_len]
#         tgt_mask = tgt_mask + [0] * (max_len - len(tgt_mask))[:max_len]
        
#         input_ids.append(src_ids)
#         target_ids.append(tgt_ids)
#         input_attention_masks.append(src_mask)
#         target_attention_masks.append(tgt_mask)
    
#     return (
#         torch.tensor(input_ids, dtype=torch.long),
#         torch.tensor(target_ids, dtype=torch.long),
#         torch.tensor(input_attention_masks, dtype=torch.long).unsqueeze(1).unsqueeze(2),
#         torch.tensor(target_attention_masks, dtype=torch.long).unsqueeze(1).unsqueeze(2)
#     )


# input_ids, target_ids,input_mask, target_mask = prepare_data(data, vocab)

# # Create TensorDataset and DataLoader
# dataset = TensorDataset(input_ids, target_ids)
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [9]:
import re
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

# Define special tokens
SPECIAL_TOKENS = {
    "<PAD>": 0,
    "<SOS>": 1,
    "<EOS>": 2,
    "<UNK>": 3,
}

# Tokenizer: Byte Pair Encoding
def get_initial_vocab(corpus):
    """
    Create an initial vocabulary with all unique characters.
    """
    vocab = defaultdict(int)
    for sentence in corpus:
        for word in sentence.split():
            vocab[" ".join(list(word)) + " </w>"] += 1  # Add word boundary
    return vocab

def merge_vocab(vocab):
    """
    Find the most frequent pair and merge it into a new token.
    """
    pairs = Counter()
    for word, freq in vocab.items():
        tokens = word.split()
        for i in range(len(tokens) - 1):
            pairs[(tokens[i], tokens[i + 1])] += freq

    if not pairs:
        return None

    most_common = max(pairs, key=pairs.get)
    return most_common

def apply_merge_rule(vocab, pair):
    """
    Apply the merge rule to the vocabulary.
    """
    new_vocab = {}
    bigram = " ".join(pair)
    replacement = "".join(pair)

    for word, freq in vocab.items():
        new_word = re.sub(rf"{re.escape(bigram)}", replacement, word)
        new_vocab[new_word] = freq

    return new_vocab

def byte_pair_encoding(corpus, num_merges=20):
    """
    Perform BPE on the given corpus and return the vocabulary.
    """
    vocab = get_initial_vocab(corpus)
    for _ in range(num_merges):
        pair = merge_vocab(vocab)
        if not pair:
            break
        vocab = apply_merge_rule(vocab, pair)
    return vocab

# Generate BPE vocabulary
all_texts = [src for src, tgt in data] + [tgt for src, tgt in data]
bpe_vocab = byte_pair_encoding(all_texts, num_merges=150)

# Map tokens to unique IDs
def build_vocab_from_bpe(bpe_vocab):
    vocab = dict(SPECIAL_TOKENS)  # Start with special tokens
    index = len(SPECIAL_TOKENS)
    for token in bpe_vocab.keys():
        for sub_token in token.split():
            if sub_token not in vocab:
                vocab[sub_token] = index
                index += 1
    return vocab

vocab = build_vocab_from_bpe(bpe_vocab)
print("length_vocab_bpe:", len(vocab))

# Tokenization
def tokenize_with_bpe(text, vocab):
    """
    Tokenize text using BPE rules and convert to IDs.
    """
    words = text.split()
    tokenized = []
    for word in words:
        chars = " ".join(list(word)) + " </w>"
        for pair in sorted(vocab, key=len, reverse=True):
            chars = chars.replace(pair, pair.replace(" ", ""))
        tokenized.extend(chars.split())
    return [vocab.get(token, vocab["<UNK>"]) for token in tokenized]

# Padding and preparation
def pad_sequences(sequences, max_len, pad_token_id):
    return np.array([
        seq + [pad_token_id] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len]
        for seq in sequences
    ])

# Prepare data
def prepare_data(data, vocab, max_len=512):
    input_ids, target_ids = [], []

    for src, tgt in data:
        # Tokenize and encode source and target
        src_ids = tokenize_with_bpe(src, vocab)
        tgt_ids = tokenize_with_bpe(tgt, vocab)

        # Add special tokens
        src_ids = [vocab["<SOS>"]] + src_ids + [vocab["<EOS>"]]
        tgt_ids = [vocab["<SOS>"]] + tgt_ids + [vocab["<EOS>"]]

        # Pad sequences
        src_ids = src_ids + [vocab["<PAD>"]] * (max_len - len(src_ids))
        tgt_ids = tgt_ids + [vocab["<PAD>"]] * (max_len - len(tgt_ids))

        input_ids.append(src_ids[:max_len])
        target_ids.append(tgt_ids[:max_len])

    return (
        torch.tensor(input_ids, dtype=torch.long),
        torch.tensor(target_ids, dtype=torch.long),
    )

input_ids, target_ids = prepare_data(data, vocab)

# Create DataLoader
dataset = TensorDataset(input_ids, target_ids)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


length_vocab_bpe: 3079


In [10]:
# vocab

In [11]:
# input_ids[input_ids >= src_vocab_size] = 0  # Replace invalid indices with <PAD>
# target_ids[target_ids >= tgt_vocab_size] = 0  # Replace invalid indices with <PAD>

# # Convert data to PyTorch tensors
# input_ids = torch.tensor(input_ids, dtype=torch.long)
# target_ids = torch.tensor(target_ids, dtype=torch.long)
# input_attention_masks = torch.tensor(input_attention_masks, dtype=torch.float32)
# target_attention_masks = torch.tensor(target_attention_masks, dtype=torch.float32)

# # Create TensorDataset and DataLoader
# dataset = TensorDataset(input_ids, target_ids, input_attention_masks, target_attention_masks)
# train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [12]:
# # Update your training loop
# for src, tgt, src_mask, tgt_mask in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
#     src, tgt = src.to(device), tgt.to(device)
#     src_mask, tgt_mask = src_mask.to(device), tgt_mask.to(device)
    
#     # Prepare target input and target output
#     tgt_input = tgt[:, :-1]
#     tgt_output = tgt[:, 1:]
#     tgt_input_mask = tgt_mask[:, :, :-1, :]  # Adjust mask dimensions
    
#     optimizer.zero_grad()
    
#     # Forward pass with masks
#     output = model(src, tgt_input, src_mask, tgt_input_mask)
#     output = output.view(-1, tgt_vocab_size)
#     tgt_output = tgt_output.contiguous().view(-1)
    
    # Rest of the training loop remains the same

# # Split data into training and validation sets
# train_inputs, val_inputs, train_targets, val_targets, train_masks, val_masks = train_test_split(
#     input_ids, target_ids, input_attention_masks, test_size=0.2, random_state=42
# )

# # Print examples
# print("Vocabulary:", vocab)
# print("\nInput IDs (Train):", train_inputs)
# print("\nTarget IDs (Train):", train_targets)
# print("\nAttention Masks_in (Train):", train_masks)
# print("\nAttention Masks_tar (Train):", train_masks)


# # Convert data to PyTorch tensors
# input_ids = torch.tensor(input_ids, dtype=torch.long)
# target_ids = torch.tensor(target_ids, dtype=torch.long)
# input_attention_masks = torch.tensor(input_attention_masks, dtype=torch.float32)
# target_attention_masks = torch.tensor(target_attention_masks, dtype=torch.float32)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from tqdm import tqdm

# Rotary Positional Encoding (RoPE) definition
class RotaryPositionalEncoding(nn.Module):
    def __init__(self, dim, max_seq_len=5):
        super().__init__()
        self.dim = dim
        self.max_seq_len = max_seq_len
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim // 2).float() / (dim // 2)))
        self.register_buffer('inv_freq', inv_freq)

    def get_pos_emb(self, seq_len, device):
        t = torch.arange(seq_len, device=device).type_as(self.inv_freq)
        freqs = torch.einsum('i,j->ij', t, self.inv_freq)
        return freqs

    def apply_rotary_pos_emb(self, x, seq_len):
        actual_seq_len = x.size(-2)
        freqs = self.get_pos_emb(actual_seq_len, x.device)
        freqs = freqs.unsqueeze(0).unsqueeze(0)
        freqs = freqs.expand(x.shape[0], x.shape[1], -1, -1)
        x_half = x[..., :self.dim//2]
        x_half_2 = x[..., self.dim//2:]
        cos = torch.cos(freqs)
        sin = torch.sin(freqs)
        x_rot = x_half * cos - x_half_2 * sin
        x_rot_2 = x_half_2 * cos + x_half * sin
        x_out = torch.cat([x_rot, x_rot_2], dim=-1)
        return x_out

# Multi-Head Attention definition
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super().__init__()
        assert embedding_dim % num_heads == 0, "embedding_dim must be divisible by num_heads"
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dim_perhead = embedding_dim // num_heads
        self.W_q = nn.Linear(embedding_dim, embedding_dim)
        self.W_k = nn.Linear(embedding_dim, embedding_dim)
        self.W_v = nn.Linear(embedding_dim, embedding_dim)
        self.W_o = nn.Linear(embedding_dim, embedding_dim)
        self.rope = RotaryPositionalEncoding(self.dim_perhead)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        seq_len = Q.size(-2)
        Q = self.rope.apply_rotary_pos_emb(Q, seq_len)
        K = self.rope.apply_rotary_pos_emb(K, seq_len)
        K_t = K.transpose(-2, -1)
        # Added small epsilon to prevent division by zero
        scale = math.sqrt(self.dim_perhead) + 1e-8
        scores = torch.matmul(Q, K_t) / scale
        
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)  # Changed from float('-inf')
        
        attention_weights = F.softmax(scores, dim=-1)
        return torch.matmul(attention_weights, V)

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.dim_perhead).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, dim_perhead = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.embedding_dim)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

# Position-wise Feedforward definition
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

# Encoder Layer definition
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

# Decoder Layer definition
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

# Transformer model definition
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length, device=src.device), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.encoder_embedding(src))
        tgt_embedded = self.dropout(self.decoder_embedding(tgt))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

   



In [14]:
# Model training setup
src_vocab_size = len(vocab)
tgt_vocab_size = len(vocab)
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 512
dropout = 0.1
num_epochs = 30

model = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

criterion = nn.CrossEntropyLoss(ignore_index=0)
# Modified optimizer parameters
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for src, tgt in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        src, tgt = src.to(device), tgt.to(device)

        # Prepare target input and target output
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()
        
        # Forward pass
        output = model(src, tgt_input)
        output = output.view(-1, tgt_vocab_size)
        tgt_output = tgt_output.contiguous().view(-1)

        # Calculate loss
        loss = criterion(output, tgt_output)
        
        # Skip if loss is NaN
        if not torch.isnan(loss):
            # Backward pass
            loss.backward()

            # Gradient clipping with smaller max_norm
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

            optimizer.step()
            epoch_loss += loss.item()
        else:
            print("NaN loss detected, skipping batch")

    print(f"Epoch {epoch+1} Loss: {epoch_loss / len(train_loader)}")

Using device: cuda


Epoch 1/30: 100%|██████████| 125/125 [00:34<00:00,  3.60it/s]


Epoch 1 Loss: 3.2229496536254882


Epoch 2/30: 100%|██████████| 125/125 [00:35<00:00,  3.56it/s]


Epoch 2 Loss: 2.4585368366241456


...

Epoch 28 Loss: 0.7334282307624816


Epoch 29/30: 100%|██████████| 125/125 [00:37<00:00,  3.33it/s]


Epoch 29 Loss: 0.6792479076385498


Epoch 30/30: 100%|██████████| 125/125 [00:37<00:00,  3.33it/s]

Epoch 30 Loss: 0.6245295598506927


In [23]:

# Load dataset
dataset_test = load_dataset("cnn_dailymail", "3.0.0", split="test[6:10]")

print(dataset_test)

# Extract articles and highlights
test_data = [(entry['article'], entry['highlights']) for entry in dataset_test]

# Prepare the test data (same as train data preparation)
test_input_ids, test_target_ids = prepare_data(test_data, vocab)

# Create DataLoader for test data
test_dataset = TensorDataset(test_input_ids, test_target_ids)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 4
})


In [24]:
# Create reverse vocab mapping (ID -> Token)
reverse_vocab = {v: k for k, v in vocab.items()}


In [25]:


def decode_ids_to_text(ids, reverse_vocab, target_length=None):
    """
    Decode token IDs to text, ensuring the text length matches the target length if provided.
    """
    tokens = [reverse_vocab.get(id, "<UNK>") for id in ids if id != vocab["<PAD>"]]
    words = []
    current_word = ""

    for token in tokens:
        if token == "<SOS>":
            words.append("<SOS>")
        elif token.endswith("</w>"):  # End of a word
            current_word += token.replace("</w>", "")  # Remove </w>
            words.append(current_word)
            current_word = ""  # Reset for the next word
        else:
            current_word += token  # Continue building the current word

    if current_word:  # Catch any final word
        words.append(current_word)

    # Ensure the length matches the target length
    if target_length is not None:
        # Join words into a single string
        words_text = " ".join(words)
        # Truncate if too long
        if len(words_text) > target_length:
            words_text = words_text[:target_length]
        # Pad if too short
        elif len(words_text) < target_length:
            words_text = words_text.ljust(target_length)  # Add spaces to reach the target length
        return words_text

    return " ".join(words)


# Evaluation loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to track gradients during testing
    for src, tgt in tqdm(test_loader, desc="Testing"):
        src, tgt = src.to(device), tgt.to(device)
        # Prepare target input and target output
        tgt_input = tgt[:, :-1]  # Shifted target for input
        tgt_output = tgt[:, 1:]  # Actual target output (without <SOS>)
        # Forward pass
        output = model(src, tgt_input)  # Get model's output
        # Get the predicted token IDs (argmax to pick the most probable token)
        pred_ids = output.argmax(dim=-1)  # Select the token with the highest probability

        # Decode the predicted and actual target IDs to text
        for i in range(pred_ids.size(0)):  # Loop over each sample in the batch
            # Calculate the length of the target sequence for this sample
            target_length = len(decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab))
            pred_text = decode_ids_to_text(pred_ids[i].cpu().numpy(), reverse_vocab, target_length=target_length)
            tgt_text = decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab)
            print(f"Source: {decode_ids_to_text(src[i].cpu().numpy(), reverse_vocab)}")
            print("********************************")
            print(f"Prediction: {pred_text}")
            print("********************************")
            
            print(f"Ground Truth: {tgt_text}")
            print("-" * 80)


Testing: 100%|██████████| 1/1 [00:00<00:00,  7.14it/s]

Source: <SOS> (CNN)Governments around the world are using the threat of terrorism -- real or perceived -- to advance executions, Amnesty International alleges in its annual report on the death penalty. "The dark trend of governments using the death penalty in a futile attempt to tackle real or imaginary threats to state security and public safety was stark last year," said Salil Shetty, Amnesty's Secretary General in a release. "It is shameful that so many states around the world are essentially playing with people's li
********************************
Prediction: Nuoesty   un e r Siath iaoolay selort forcr nu ovganrtgevg thgne  Gut .tt alk  .n potb r  of iheu  heaaence  to aiftt I Cneensaelion alaims toat wooernment  aseugt aor farkd tse apedg aoe rhreets.f rhrrorigm .o t vo ie .apcut on  o Mhe petbersof Sxpcut ons oirld ape tav cott puendnu t sost .0 ..lnmaned .ith .00   .et dulth .tatedced .r ie p0 al <EOS>oeeeee
********************************
Ground Truth: <SOS> Amnesty's annual 

In [26]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Evaluation loop
model.eval()  # Set the model to evaluation mode
rouge_scores = {"rouge1": [], "rouge2": [], "rougeL": []}

with torch.no_grad():  # No need to track gradients during testing
    for src, tgt in tqdm(test_loader, desc="Testing"):
        src, tgt = src.to(device), tgt.to(device)
        # Prepare target input and target output
        tgt_input = tgt[:, :-1]  # Shifted target for input
        tgt_output = tgt[:, 1:]  # Actual target output (without <SOS>)
        # Forward pass
        output = model(src, tgt_input)  # Get model's output
        # Get the predicted token IDs (argmax to pick the most probable token)
        pred_ids = output.argmax(dim=-1)  # Select the token with the highest probability

        # Decode the predicted and actual target IDs to text
        for i in range(pred_ids.size(0)):  # Loop over each sample in the batch
            # Calculate the length of the target sequence for this sample
            target_length = len(decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab))
            pred_text = decode_ids_to_text(pred_ids[i].cpu().numpy(), reverse_vocab, target_length=target_length)
            tgt_text = decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab)
            
            # Calculate ROUGE scores
            scores = scorer.score(tgt_text, pred_text)
            rouge_scores["rouge1"].append(scores["rouge1"].fmeasure)
            rouge_scores["rouge2"].append(scores["rouge2"].fmeasure)
            rouge_scores["rougeL"].append(scores["rougeL"].fmeasure)

            # Print for debugging
            print(f"Source: {decode_ids_to_text(src[i].cpu().numpy(), reverse_vocab)}")
            print(f"Prediction: {pred_text}")
            print(f"Ground Truth: {tgt_text}")
            print("-" * 80)

# Calculate and print average ROUGE scores
avg_rouge1 = sum(rouge_scores["rouge1"]) / len(rouge_scores["rouge1"])
avg_rouge2 = sum(rouge_scores["rouge2"]) / len(rouge_scores["rouge2"])
avg_rougeL = sum(rouge_scores["rougeL"]) / len(rouge_scores["rougeL"])

print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")


Testing: 100%|██████████| 1/1 [00:00<00:00,  6.70it/s]

Source: <SOS> (CNN)Governments around the world are using the threat of terrorism -- real or perceived -- to advance executions, Amnesty International alleges in its annual report on the death penalty. "The dark trend of governments using the death penalty in a futile attempt to tackle real or imaginary threats to state security and public safety was stark last year," said Salil Shetty, Amnesty's Secretary General in a release. "It is shameful that so many states around the world are essentially playing with people's li
Prediction: Nuoesty   un e r Siath iaoolay selort forcr nu ovganrtgevg thgne  Gut .tt alk  .n potb r  of iheu  heaaence  to aiftt I Cneensaelion alaims toat wooernment  aseugt aor farkd tse apedg aoe rhreets.f rhrrorigm .o t vo ie .apcut on  o Mhe petbersof Sxpcut ons oirld ape tav cott puendnu t sost .0 ..lnmaned .ith .00   .et dulth .tatedced .r ie p0 al <EOS>oeeeee
Ground Truth: <SOS> Amnesty's annual death penalty report catalogs encouraging signs, but setbacks in n

In [ ]:
def decode_ids_to_text(ids, reverse_vocab):
    tokens = [reverse_vocab.get(id, "<UNK>") for id in ids if id != vocab["<PAD>"]]
    words = []
    current_word = ""

    for token in tokens:
        if token == "<SOS>":
            words.append("<SOS>")
        elif token.endswith("</w>"):  # End of a word
            current_word += token.replace("</w>", "")  # Remove </w>
            words.append(current_word)
            current_word = ""  # Reset for the next word
        else:
            current_word += token  # Continue building the current word
    
    if current_word:  # Catch any final word
        words.append(current_word)
    
    return " ".join(words)


# Evaluation loop
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to track gradients during testing
    for src, tgt in tqdm(test_loader, desc="Testing"):
        src, tgt = src.to(device), tgt.to(device)
        # Prepare target input and target output
        tgt_input = tgt[:, :-1]  # Shifted target for input
        tgt_output = tgt[:, 1:]  # Actual target output (without < SOS >)
        # Forward pass
        output = model(src, tgt_input)  # Get model's output
        # Get the predicted token IDs (argmax to pick the most probable token)
        pred_ids = output.argmax(dim=-1)  # Select the token with the highest probability
        # Decode the predicted and actual target IDs to text
        for i in range(pred_ids.size(0)):  # Loop over each sample in the batch
            pred_text = decode_ids_to_text(pred_ids[i].cpu().numpy(), reverse_vocab)
            tgt_text = decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab)
            print(f"Source: {decode_ids_to_text(src[i].cpu().numpy(), reverse_vocab)}")
            print(f"Prediction: {pred_text}")
            print(f"Ground Truth: {tgt_text}")
            print("-" * 80)


In [23]:
# # Function to decode token IDs to text
# def decode_ids_to_text(ids, reverse_vocab):
#     return " ".join([reverse_vocab.get(id, "<UNK>") for id in ids if id != vocab["<PAD>"]])

# # Evaluation loop
# model.eval()  # Set the model to evaluation mode

# with torch.no_grad():  # No need to track gradients during testing
#     for src, tgt in tqdm(test_loader, desc="Testing"):
#         src, tgt = src.to(device), tgt.to(device)

#         # Prepare target input and target output
#         tgt_input = tgt[:, :-1]  # Shifted target for input
#         tgt_output = tgt[:, 1:]  # Actual target output (without <SOS>)

#         # Forward pass
#         output = model(src, tgt_input)  # Get model's output

#         # Get the predicted token IDs (argmax to pick the most probable token)
#         pred_ids = output.argmax(dim=-1)  # Select the token with the highest probability

#         # Decode the predicted and actual target IDs to text
#         for i in range(pred_ids.size(0)):  # Loop over each sample in the batch
#             pred_text = decode_ids_to_text(pred_ids[i].cpu().numpy(), reverse_vocab)
#             tgt_text = decode_ids_to_text(tgt[i].cpu().numpy(), reverse_vocab)

#             print(f"Source: {' '.join([reverse_vocab.get(idx, '<UNK>') for idx in src[i].cpu().numpy()])}")
#             print(f"Prediction: {pred_text}")
#             print(f"Ground Truth: {tgt_text}")
#             print("-" * 80)


Testing: 100%|██████████| 2/2 [00:00<00:00, 11.62it/s]

Source: <SOS> LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they
********************************
Prediction: Narry Potter star daniel Hadcliffe gets f20M fortune as he thrns M8 donday . Toung rttor says he has no plans to frotter his cash aday . <EOS>adcliffe s earnings hrom first fuve ootter tilms have been deld in thast fond . <EOS>ae
********************************
Ground Truth: <SOS> Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe